In [ ]:
#load libraries 
import numpy as np
import scipy.sparse as sp
import cplex as cp

In [ ]:
def linear_programming(direction, A, senses, b, c, l, u):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(), col_indices.tolist(), A.data.tolist()))

    # solve the problem
    prob.solve()

    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)

In [ ]:
def assignment_problem(filename):
    # read problem instance from file
    problem = np.loadtxt(filename)

    N = np.sqrt(problem.shape[0]).astype(int)

    c = problem[:, 2]
    senses = np.repeat("E", 2 * N)
    b = np.repeat(1, 2 * N)
    l = np.repeat(0, N * N)
    u = np.repeat(1, N * N)
    types = np.repeat("B", N * N)

    aij = np.repeat(1.0, 2 * N * N)
    row = np.repeat(range(2 * N), N)
    col = np.concatenate((range(N * N), 
                          np.reshape(np.array(range(N * N)).reshape((N, N)), (N * N,), order = "F")))
    A = sp.csr_matrix((aij, (row, col)), shape = (2 * N, N * N))

    x_star, obj_star = mixed_integer_linear_programming("maximize", A, senses, b, c, l, u, types)
    return(x_star, obj_star)

In [ ]:
D = 3
S = 2
